To verify the saved model checkpoint, we will load the model weights and test it on validation data.

In [1]:
import keras
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
import numpy as np
import pandas as pd
from keras import layers
import os
from keras import regularizers
import math
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.preprocessing.sequence import pad_sequences
%set_env AWS_PROFILE=uaprofile
from sensorfabric.athena import athena
import time
from datetime import datetime, timedelta
db = athena(database='elise', offlineCache=True)

2023-11-29 14:45:54.533632: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-29 14:45:54.712109: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 14:45:55.439114: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: AWS_PROFILE=uaprofile


In [42]:
#keras.utils.set_random_seed(912)

# 45 spontaneous PIDs having at least 21 days of temperature data prior to labor date.
# FOLD 
train_pids = [15, 67, 80, 161, 183, 189, 1, 2, 7, 10, 11, 29, 35, 36, 48, 55, 61, 69, 72, 73, 75, 76, 77, 88, 90, 91, 93, 97, 99, 102, 
              104, 106, 107, 108, 110, 111, 113, 120, 124, 126, 129, 136, 138, 141, 144, 152, 156, 170 ]

test_pids = [173, 179, 191, 193, 195, 197]

data_dir = '/home/chinmai/src/Oura/Data/ConvAE_LinIP_Encodings_40LD_y'
#exp_dir  = '/home/chinmai/src/Oura/Data/Linear_Interpolation_y/'

train_pids.sort()
test_pids.sort()
print(train_pids, test_pids)
print(len(train_pids),len(test_pids))

# We want to further divide the training PIDs to training and Validation PIDs
train_pids,val_pids = train_test_split(train_pids, test_size = 0.1, random_state=42)
val_pids.sort()
print(train_pids, val_pids)
print(len(train_pids), len(val_pids))

[1, 2, 7, 10, 11, 15, 29, 35, 36, 48, 55, 61, 67, 69, 72, 73, 75, 76, 77, 80, 88, 90, 91, 93, 97, 99, 102, 104, 106, 107, 108, 110, 111, 113, 120, 124, 126, 129, 136, 138, 141, 144, 152, 156, 161, 170, 183, 189] [173, 179, 191, 193, 195, 197]
48 6
[129, 67, 80, 11, 99, 36, 10, 29, 138, 113, 69, 76, 170, 73, 48, 75, 107, 111, 183, 1, 110, 108, 15, 61, 120, 2, 161, 90, 7, 126, 124, 93, 144, 55, 91, 77, 189, 88, 35, 152, 72, 106, 136] [97, 102, 104, 141, 156]
43 5


In [43]:
# In this block we want to read from training and test PIDs
# Reading the ConvAE encodings as input and days to labor as output.
x_train = []
y_train = []
count = 0
mlen = 0
for pid in train_pids:
    fname = os.path.join(data_dir,str(pid)+'_5temp_encoding_40LD.csv')
    #print('Processing pid: ',pid)
    data = np.loadtxt(fname,delimiter=',')
    d1 = data[:,0:64]
    d2 = data[:,-1]
    mr,mc = data.shape
    if (mr > mlen):
        mlen = mr
    x_train.append(d1)
    y_train.append(d2)
    count += 1

x_val = []
y_val = []
count = 0
for pid in val_pids:
    fname = os.path.join(data_dir,str(pid)+'_5temp_encoding_40LD.csv')
    #print('Processing pid: ',pid)
    data = np.loadtxt(fname,delimiter=',')
    d1 = data[:,0:64]
    d2 = data[:,-1]
    if (mr > mlen):
        mlen = mr
    x_val.append(d1)
    y_val.append(d2)
    count += 1

x_test = []
y_test = []
count = 0
for pid in test_pids:
    fname = os.path.join(data_dir,str(pid)+'_5temp_encoding_40LD.csv')
    #print('Processing pid: ',pid)
    data = np.loadtxt(fname,delimiter=',')
    d1 = data[:,0:64]
    d2 = data[:,-1]
    if (mr > mlen):
        mlen = mr
    x_test.append(d1)
    y_test.append(d2)
    count += 1

#print(mlen)

# Maximum length will no longer be 40. It is infact 52
x_tr_pad = pad_sequences(x_train, dtype='float32', maxlen=mlen, padding='post')
y_tr_pad = pad_sequences(y_train, dtype='float32', maxlen=mlen, padding='post')
x_tr = np.asarray(x_tr_pad)
y_tr = np.asarray(y_tr_pad)

a,b = y_tr.shape
y_tr = np.reshape(y_tr,(a,b,1))

    
x_va_pad = pad_sequences(x_val, dtype='float32', maxlen=mlen, padding='post')
y_va_pad = pad_sequences(y_val, dtype='float32', maxlen=mlen, padding='post')
x_va = np.asarray(x_va_pad)
y_va = np.asarray(y_va_pad)
a,b = y_va.shape
y_va = np.reshape(y_va,(a,b,1))


x_te_pad = pad_sequences(x_test, dtype='float32', maxlen=mlen, padding='post')
y_te_pad = pad_sequences(y_test, dtype='float32', maxlen=mlen, padding='post')

x_te = np.asarray(x_te_pad)
y_te = np.asarray(y_te_pad)

a,b = y_te.shape
y_te = np.reshape(y_te,(a,b,1))


print(x_tr.shape, x_va.shape, x_te.shape)
print(y_tr.shape, y_va.shape, y_te.shape)


(43, 52, 64) (5, 52, 64) (6, 52, 64)
(43, 52, 1) (5, 52, 1) (6, 52, 1)


In [44]:
keras.utils.set_random_seed(912)
# LSTM model
input_img = keras.Input(shape=[mlen,64])
m1 = layers.Masking(mask_value = 0)(input_img)

l1 = layers.LSTM(128,activation = 'tanh', use_bias=True, kernel_initializer="glorot_uniform",
                 recurrent_initializer="orthogonal", bias_initializer="zeros", return_sequences = True, unroll = True)
#nl1 = layers.LayerNormalization(axis=1)(l1)
x = l1(m1)
ln1 = layers.LayerNormalization(axis=1)(x)
d0 = layers.Dense(128,activation='LeakyReLU', kernel_initializer='glorot_uniform')(ln1)
# LSTM 
d1 = layers.Dense(1,activation='linear', kernel_initializer='glorot_uniform')(d0)

#print(x.shape)
#print(d1.shape)

In [45]:
keras.utils.set_random_seed(912)
simple_lstm = keras.Model(input_img,d1)
simple_lstm.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 52, 64)]          0         
                                                                 
 masking_4 (Masking)         (None, 52, 64)            0         
                                                                 
 lstm_4 (LSTM)               (None, 52, 128)           98816     
                                                                 
 layer_normalization_4 (Lay  (None, 52, 128)           104       
 erNormalization)                                                
                                                                 
 dense_8 (Dense)             (None, 52, 128)           16512     
                                                                 
 dense_9 (Dense)             (None, 52, 1)             129       
                                                           

In [46]:
keras.utils.set_random_seed(912)
# MAE loss - mean(abs(true Temperature - reconstructed Temprature from encoding))
simple_lstm.compile(optimizer=keras.optimizers.Adam(), loss='mean_absolute_error')
simple_lstm.load_weights('./fold9_40LD/final_weights.keras')

In [47]:
# Run the Evaluate function on the test dataset.
res = simple_lstm.evaluate(x_va,y_va)
res = simple_lstm.evaluate(x_te,y_te)

1/1 [==============================] - 0s 16ms/step - loss: 5.2400


In [48]:
#res = simple_lstm.evaluate(x_te,y_te)
pred = simple_lstm.predict(x_te)

1/1 [==============================] - 1s 514ms/step


In [49]:
# Length of test set
te_len = len(test_pids)
err_list = []
err_avg_list = []
for i in range(0,te_len):

    err = []
    trim_0 = [x for x in y_te[i] if x!= 0]
    print(len(trim_0))
    
    days = len(trim_0)
    for j in range(0,days):
        err.append(abs(float(y_te[i][j] - pred[i][j])))

    #print(days)
    #print(pred.shape)
    print(err)
    avg = sum(err)/len(err)
    err_list.append(err)
    err_avg_list.append(avg)

25
[12.254083633422852, 9.986349105834961, 11.946474075317383, 11.159368515014648, 9.401437759399414, 8.756692886352539, 10.50056266784668, 6.639898300170898, 9.764425277709961, 9.599466323852539, 6.015737533569336, 7.069204330444336, 7.360784530639648, 5.826555252075195, 6.824136734008789, 7.385988235473633, 0.5139942169189453, 1.377302646636963, 0.32809972763061523, 2.6384990215301514, 2.0540270805358887, 2.5589375495910645, 0.7432856559753418, 0.0007073879241943359, 3.2909746170043945]
34
[2.3505306243896484, 2.784198760986328, 0.5671100616455078, 0.3721961975097656, 0.5926876068115234, 0.11930274963378906, 0.7735309600830078, 0.9791412353515625, 2.7908992767333984, 3.3972854614257812, 3.322225570678711, 1.9994487762451172, 2.639925003051758, 1.989328384399414, 0.33205604553222656, 1.131692886352539, 0.9049148559570312, 1.7312631607055664, 1.3524885177612305, 0.845942497253418, 1.6649904251098633, 1.0862865447998047, 0.22890186309814453, 2.1606645584106445, 2.8852953910827637, 2.311

In [50]:
print(err_avg_list)
print(sum(err_avg_list)/len(err_avg_list))

[6.159879722595215, 1.5666304195628447, 3.197671687602997, 10.712097108364105, 5.502992593325102, 2.67346613747733]
4.968789611487932


In [51]:
# PIDs with EDD error.
edd_err_pids = [71, 72, 75, 123, 138]

# Writing Individual CSV Files to FOLD folder.
# Length of test set
te_len = len(test_pids)
err_list = []
err_avg_list = []
# Flag to indicate Before and After EDD
before = 0

for i in range(0,te_len):
    p = str(test_pids[i])
    print('PID: ', p)
    fname = './Spontaneous_Model_Predictions_40LD/'+str(p)+'.csv'
    #print(fname)
    fd = open(fname,'w')
    
    # 1st Line is the PID
    fd.write('PID '+ p +'\n')
    
    # First: We need to calculate the relative distance from EDD
    # Get the labor date for the PID from the enroll_labor table
    query2 = 'select edd, labordate from edd_laboronset where pid = {pid}'
    res2   = db.execQuery(query2.format(pid=p), cached=True)
    
    labordate = pd.to_datetime(res2['labordate'][0])
    #print ('Labor date for pid: ',p,' is ',labordate)

    edd = pd.to_datetime(res2['edd'][0])
    #print ('EDD for pid: ',p,' is ',edd)
    
     # Make the Error correction in EDD
    if pid in edd_err_pids:
        edd = edd + timedelta(days = 365)
        #print('EDD error corrected for pid ',pid,'. EDD is ',edd)

    # Calculate the difference between EDD and laboronset.
    if edd < labordate:
        # Before = 0
        # AFTER EDD
        # Get the difference in days from EDD to laboronset.
        dd = (labordate - edd).days
        val = 40 + dd
        print ('After:',dd, val)
        
    else:        # This means that EDD is >= Labordate
        # BEFORE
        before = 1
        # Get the difference in days from EDD to laboronset.
        dd = (edd-labordate).days
        val = 40-dd
        print ('Before:',dd, val)
    
    
    # Remove all zeros from y_te and pred[i] and reshapee pred[i] to be a vector
    trim_y_te = [float(x) for x in y_te[i] if x!= 0]
    trim_pred_i = pred[i][0:len(trim_y_te)]
    trim_pred_i = np.reshape(trim_pred_i,(len(trim_pred_i),))
    
    # Cieling of the first y value is the total available days of data.
    # This depends whether the labor happened before of after EDD.
    #cei_y = math.ceil(trim_y_te[0])
    #print(cei_y)
    
    fd.write('Days Before Labor ,')
    for j in range(len(trim_y_te),1,-1):    # Write from 240 to maximum length of sequence.
        fd.write(str(j)+',')
    fd.write(str(j-1)+'\n')
    
    
    # 3rd Line - Ground Truth
    fd.write('Ground Truth ,')
    for j in range(0,len(trim_y_te)-1):
        fd.write(str(float(trim_y_te[j]))+',')
    fd.write(str(float(trim_y_te[j+1]))+'\n')

    # 4th Line - Ground Truth
    fd.write('Model Prediction ,')
    for j in range(0,len(trim_pred_i)-1):
        fd.write(str(float(trim_pred_i[j]))+',')
    fd.write(str(float(trim_pred_i[j+1]))+'\n')
    
    err = []
    abs_err = []
    days = len(trim_y_te)
    for j in range(0,days):
        abs_err.append(abs(float(trim_y_te[j] - trim_pred_i[j])))
        err.append(float(trim_y_te[j] - trim_pred_i[j]))

    # 5th Line
    fd.write('Error ,')
    for j in range(0,len(err)-1):
        fd.write(str(float(err[j]))+',')
    fd.write(str(float(err[j+1]))+'\n')

    # 6th Line
    fd.write('Absolute Error ,')
    for j in range(0,len(abs_err)-1):
        fd.write(str(float(abs_err[j]))+',')
    fd.write(str(float(abs_err[j+1]))+'\n')

    # 7th Line
    fd.write('Mean Absolute Error ,')
    fd.write(str(float(sum(abs_err)/len(abs_err)))+'\n')

    fd.close()
    
    #print(days)
    #print(pred.shape)
    #print(err)
    #avg = sum(err)/len(err)
    #err_list.append(err)
    #err_avg_list.append(avg)


PID:  173
Before: 0 40
PID:  179
Before: 6 34
PID:  191
Before: 0 40
PID:  193
After: 12 52
PID:  195
Before: 2 38
PID:  197
After: 2 42


In [ ]:
print(len(act_pred))
print(y_tr.shape)
for i in range(0,len(act_pred)): 
    x = act_pred[i] - y_tr[i]
    print(x)
    
#print(y_te)